Versión renderizada e interactiva: https://nbviewer.jupyter.org/github/marcgror/COVID-19-Data-Analysis/blob/master/COVID.ipynb

# Análisis de los datos del COVID-19 en España
2020 será siempre recordado como el año de la pandemia de COVID-19. España ha sido y es uno de los países más afectados, tanto en la primera ola en marzo como en la siguiente en setiembre.

Para analizar la situación, a nivel nacional, autonómico y provincial, se usarán los datos proporcionados por el Ministerio de Sanidad.

In [26]:
# Import required packages
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from ipywidgets import widgets
import matplotlib.dates as mdates
import chart_studio.plotly as py
import chart_studio.tools as tls
sns.set_style(style="darkgrid")
from datawrapper import Datawrapper

In [27]:
from funciones import create_interactive_plot

In [28]:
# Load each data file from the repository
ccaa = pd.read_csv('ccaa.csv', parse_dates=True)
#provincias = pd.read_csv('https://raw.githubusercontent.com/montera34/escovid19data/master/data/output/covid19-provincias-spain_consolidated.csv')
spain = pd.read_csv('spain.csv', parse_dates=True)

# España 

## Métodos de diagnóstico del COVID-19
En España, el COVID se está detectando principalmente usando las pruebas de PCR y test de anticuerpos, a parte de otras pruebas como el test de antígenos.

La prueba PCR (Polimerasa Chain Reaction) es una prueba de diagnóstico utilizada para detectar fragmentos del material genético de un patógeno. En este caso en concreto, el patógeno se corresponde con el virus SARS-CoV-2. La idea es sencilla: se toma una muestra respiratoria de la persona sospechosa de infección, y se analiza en un laboratorio de microbiología con un PCR en busca de una molécula de ARN del virus. El positivo vendría cuando la prueba detecta ese ARN del virus, mientras será negativo en caso contrario.

Actualmente se dispone tanto de datos de contagiados diagnosticados con PCR como de diagnosticados con otros tipos de prueba, como puede ser el test de anticuerpos o de antígenos.

El siguiente gráfico muestra la contribución en porcentaje de cada tipo de prueba. Como es obvio, más de un 90 % de las detecciones de contagiados son gracias a PCR.

In [29]:
labels = ['PCR', 'Test AC', 'Otras', 'Desconocida']
values = list(spain[['num_casos_prueba_pcr', 'num_casos_prueba_test_ac', 'num_casos_prueba_otras', 'num_casos_prueba_desconocida']].sum())
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=0.3)], layout=dict(title='Contribución de cada método de detección', width=500))
py.iplot(fig, filename='Test Pie Chart')

## Evolución de los contagios
En la siguiente tabla se muestran los casos diarios, acumulados, el incremento diario, en los últimos 7 y 14 días, la media de casos en los últimos 7 y 14 días, la incidéncia acumulada en los últimos 7 y 14 días y la acumulada para los últimos 10 días a nivel nacional:

In [30]:
fig = go.Figure(data=[go.Table(
    columnwidth=[100, 90, 90, 180, 100, 120, 140, 140, 100, 100, 100],
    header=dict(values=list(['Fecha', 'Casos nuevos', 'Casos totales', 'Incremento de casos', 'Casos 7 días', 'Casos 14 días', 'Media casos 7 días', 'Media casos 14 días', 'IA 7 días', 'IA 14 días', 'IA total']),
                fill_color='rgb(124,124,124)',
                align='center', font=dict(color='white')),
    cells=dict(values=spain[['fecha', 'num_casos', 'cases_accumulated', 'cases_inc', 'cases_7_days', 'cases_14_days', 'avg_cases_7_days', 'avg_cases_14_days', 'ia_100000_week', 'ia_100000_2week', 'ia_accumulated']].tail(10).transpose().values.tolist(),
               fill_color='rgb(179, 179, 179)',
               align='center', height=25))
])
fig.update_layout(width=1200, height=500, title='Datos de contagios para los últimos 10 días')
py.iplot(fig, filename='Infected table')

La curva de contagiados en España desde el inicio de la pandémia se muestra a continuación:

In [31]:
fig = go.Figure()
#fig.add_trace(go.Bar(x=spain['fecha'], y=spain['num_casos_prueba_pcr'], name= 'Nuevos PCR'))
fig.add_trace(go.Bar(x=spain['fecha'], y=spain['num_casos'], name='Nuevos casos'))
fig.add_trace(go.Scatter(x=spain['fecha'], y=spain['avg_cases_7_days'], name='Media de 7 días'))
fig.update_layout(legend=dict(orientation='h', bgcolor='LightSteelBlue'), width=1200, title='Casos diarios desde el inicio de la pandémia')
fig.add_vrect(x0='2020-03-14', x1='2020-06-21', y1=100, line_width=0, fillcolor='grey', opacity=0.2)
fig.add_annotation(text='Primer Estado Alarma', hovertext='Periodo de declaración del primer Estado de Alarma de la pandémia', x='2020-04-24', y=19000, showarrow=False)
fig.add_vline(x='2020-10-25', line_dash='dash', line_color='red')
fig.add_annotation(text='', hovertext='Declaración del segundo Estado de Alarma Nacional de la pandemia', x='2020-10-25', y=19000, showarrow=True)
py.iplot(fig, filename='Daily infected')

Las dos olas se pueden diferenciar claramente. Los efectos del confinamiento, desescalada y fin del Estado de Alarma son también evidentes, ya que el número de contagios diarios decae a mínimos practicamente hasta principios de julio, fruto del parón social y laboral del país. Es entonces cuando empiezan a aumentar los nuevos positivos, seguramente porque la "Nueva Normalidad" lleva vigente 1-2 semanas en toda España, y por lo tanto la población empieza poco a poco a interactuar de nuevo. El hecho de que esta vuelta a la socialización y al trabajo se diera en verano seguramente no ayuda tampoco.

Si nos fijamos en los positivos acumulados, las tendencias son prácticamente las mismas, como es lógico:

In [32]:
fig = go.Figure(data=[go.Bar(x=spain['fecha'], y=spain['cases_accumulated'])], layout=go.Layout(title=go.layout.Title(text='Casos PCR acumulados desde el inicio de la pandémia')))
fig.add_vrect(x0='2020-03-14', x1='2020-06-21', y1=1400000, line_width=0, fillcolor='grey', opacity=0.2)
fig.add_annotation(text='Primer Estado Alarma', hovertext='Periodo de declaración del primer Estado de Alarma de la pandémia', x='2020-04-24', y=1200000, showarrow=False)
fig.update_layout(width=1200)
py.iplot(fig, filename='Infected accumulated')

Otros indicadores útiles para valorar el estado actual de la pandemia son el número de positivos hospitalizados, en la UCI y recuperados. Como se puede observar, en el peor momento de la primera ola (finales de marzo-mayo) es cuando se produjo el mayor número de personas hospitalizadas o en la UCI, mientras que los recuperados iban aumentando a medida que pasaba las semanas. Durante el inicio del verano los ingresados disminuyen, fruto del descenso de los contagios debido al confinamiento y la desescalada. Una vez llega Agosto, se empiezan a notar los efectos de la "nueva normalidad", y los hospitalizados e ingresados en UCI por COVID empiezan a aumentar de nuevo.

Durante todo este tiempo, las cifras de recuperados no hacen más que aumentar, debido seguramente al gran número de personas infectadas con anterioridad.

### Incidéncia acumulada
La incidéncia acumulada (IA) es una forma de medir el impacto de la pandémia en cada territorio teniendo en cuenta su población. Por lo tanto, permite comparar valores entre distintas regiones/paises y establecer valores máximos que, una vez superados, exijan aplicar medidas de restricción de movilidad.

La evolución de la IA en 7 y 14 días durante toda la pandémia se muestra a continuación:

In [33]:
fig = go.Figure(data=[go.Scatter(x=spain['fecha'], y=spain['ia_100000_2week'], mode='lines', name='IA 14 días')], layout=go.Layout(title=go.layout.Title(text='Incidencia acumulada')))
fig.add_trace(go.Scatter(x=spain['fecha'], y=spain['ia_100000_week'], mode='lines', name='IA 7 días'))
fig.add_vrect(x0='2020-03-14', x1='2020-06-21', y1=580, line_width=0, fillcolor='grey', opacity=0.2)
fig.add_annotation(text='Primer Estado Alarma', hovertext='Periodo de declaración del primer Estado de Alarma de la pandémia', x='2020-04-24', y=530, showarrow=False)
fig.update_layout(width=1200, legend=dict(orientation='h', bgcolor='LightSteelBlue'))
py.iplot(fig, filename='IA 7 vs IA 14')

Se observa una tendéncia similar a la curva de contágios.

## Fallecidos

In [34]:
fig = go.Figure(data=[go.Table(
    columnwidth=[100, 180, 180, 250, 180, 180, 200, 200, 200, 250],
    header=dict(values=list(['Fecha', 'Nuevos fallecidos', 'Fallecidos totales', 'Incremento de fallecidos (%)', 'Última semana', 'Últimos 14 días', 'Media fallecidos 3 días', 'Media fallecidos 7 días', 'Por 100000 habitantes', 'Por 1M hab. en la última semana']),
                fill_color='rgb(124,124,124)',
                align='center', font=dict(color='white')),
    cells=dict(values=spain[['fecha', 'deceased', 'deceased_accumulated', 'deceased_inc', 'deaths_7_days', 'deaths_14_days', 'avg_deaths_3_days', 'avg_deaths_7_days', 'deceased_per_100000', 'deaths_7_days_1M']].tail(10).transpose().values.tolist(),
               fill_color='rgb(179, 179, 179)',
               align='center', height=25))
])
fig.update_layout(width=1500, height=500, title='Datos de fallecidos para los últimos 10 días')
py.iplot(fig, filename='deceased table')

El siguiente paso es el número de fallecidos. A continuación se muestra la evolución del número de fallecidos diario diagnosticados con COVID desde el inicio de la pandémia:

In [35]:
fig = go.Figure(data=[go.Bar(x=spain['fecha'], y=spain['deceased'], name='Fallecidos diarios')], layout=go.Layout(title=go.layout.Title(text='Fallecimientos diarios desde el inicio de la pandémia')))
fig.add_trace(go.Scatter(x=spain['fecha'], y=spain['avg_deaths_7_days'], name='Media de 7 días'))
fig.update_yaxes(range=[0,1000])
fig.add_vrect(x0='2020-03-14', x1='2020-06-21', y1=1000, line_width=0, fillcolor='grey', opacity=0.2)
fig.add_annotation(text='Primer Estado Alarma', hovertext='Periodo de declaración del primer Estado de Alarma de la pandémia', x='2020-04-24', y=900, showarrow=False)
fig.update_layout(width=1200, legend=dict(orientation='h', bgcolor='LightSteelBlue'))
py.iplot(fig, filename='Daily deaths')

Durante la primera ola, se alcanzaron cifras diarias de más de 1000 muertos diarios positivos por COVID. Gracias al confinamiento, estas cifras fueron dismuyendo lentamente durante mayo y junio, llegando a su mínimo durante el verano. Una vez los contagios e infectados vuelven a incrementarse a finales de agosto, los fallecidos empiezan a aumentar también.

In [36]:
fig = go.Figure(data=[go.Bar(x=spain['fecha'], y=spain['deceased_accumulated'])], layout=go.Layout(title=go.layout.Title(text='Fallecimientos acumulados desde el inicio de la pandémia')))
fig.add_vrect(x0='2020-03-14', x1='2020-06-21', y1=40000, line_width=0, fillcolor='grey', opacity=0.2)
fig.add_annotation(text='Primer Estado Alarma', hovertext='Periodo de declaración del primer Estado de Alarma de la pandémia', x='2020-04-24', y=35000, showarrow=False)
fig.update_layout(width=1200)
py.iplot(fig, filename='Accumulated deaths')

En el acumulado, se puede observar un estancamiento en el número de fallecidos durante el verano, pero a partir de los últimos días de agosto se observa de nuevo una tendencia al aumento.

## Ingresados y en UCI

In [37]:
fig = go.Figure(data=[go.Bar(x=spain['fecha'], y=spain['hospitalizated'], name='Hospitalizados diarios')], layout=go.Layout(title=go.layout.Title(text='Hospitalizados diarios desde el inicio de la pandémia')))
fig.add_trace(go.Scatter(x=spain['fecha'], y=spain['avg_hospitalizated_7_days'], name='Media de 7 días'))
#fig.update_yaxes(range=[0,1000])
fig.add_vrect(x0='2020-03-14', x1='2020-06-21', y1=1000, line_width=0, fillcolor='grey', opacity=0.2)
fig.add_annotation(text='Primer Estado Alarma', hovertext='Periodo de declaración del primer Estado de Alarma de la pandémia', x='2020-04-24', y=3900, showarrow=False)
fig.update_layout(width=1200, legend=dict(orientation='h', bgcolor='LightSteelBlue'))
py.iplot(fig, filename='Daily hospitalizated')

In [38]:
fig = go.Figure(data=[go.Bar(x=spain['fecha'], y=spain['UCI'], name='Ingresados en UCI diarios')], layout=go.Layout(title=go.layout.Title(text='Ingresados en UCI diarios desde el inicio de la pandémia')))
fig.add_trace(go.Scatter(x=spain['fecha'], y=spain['avg_UCI_7_days'], name='Media de 7 días'))
#fig.update_yaxes(range=[0,1000])
fig.add_vrect(x0='2020-03-14', x1='2020-06-21', y1=1000, line_width=0, fillcolor='grey', opacity=0.2)
fig.add_annotation(text='Primer Estado Alarma', hovertext='Periodo de declaración del primer Estado de Alarma de la pandémia', x='2020-04-24', y=390, showarrow=False)
fig.update_layout(width=1200, legend=dict(orientation='h', bgcolor='LightSteelBlue'))
py.iplot(fig, filename='Daily UCI')

# CCAA

## Métodos de diagnóstico del COVID-19

In [39]:
x = ccaa['ccaa'].unique().tolist()
fig = go.Figure(layout=go.Layout(title=go.layout.Title(text='Distribución de los diferentes métodos de diagnóstico para cada CCAA')))
fig.add_trace(go.Bar(y=x, x=ccaa['cases_accumulated_PCR_percentage'].tail(20), name='PCR', orientation='h'))
fig.add_trace(go.Bar(y=x, x=ccaa['cases_accumulated_AC_percentage'].tail(20), name='AC', orientation='h'))
fig.add_trace(go.Bar(y=x, x=ccaa['cases_accumulated_otras_percentage'].tail(20), name='Otras', orientation='h'))
fig.add_trace(go.Bar(y=x, x=ccaa['cases_accumulated_desconocida_percentage'].tail(20), name='Desconocido', orientation='h'))
fig.update_layout(width=1200, barmode='stack', legend=dict(orientation='h', bgcolor='LightSteelBlue'))
py.iplot(fig, filename='Test CCAA bar Chart')

A partir de los datos separados por Comunidades Autónomas es posible seguir la evolución de la pandemia en estas.

## Evolución de los contagios y fallecidos
Empecemos con una tabla resumen: en ella se muestran los casos acumulados diagnosticados por PCR, el número de nuevos casos (diarios, en 7 y en 14 días) y la IA por 100000 habitantes para cada comunidad autónoma, ordenado por IA:

In [40]:
fig = go.Figure(data=[go.Table(
    columnwidth=[60, 100, 70, 70, 70, 70, 50, 50],
    header=dict(values=list(['Fecha', 'CCAA', 'Casos totales', 'Casos diarios', 'Casos 7 días', 'Casos 14 días', 'IA 7 días', 'IA 14 días']),
                fill_color='rgb(124,124,124)',
                align='center', font=dict(color='white')),
    cells=dict(values=ccaa[['fecha', 'ccaa','cases_accumulated', 'num_casos', 'cases_7_days', 'cases_14_days', 'ia_100000_week', 'ia_100000_2week']].tail(20).sort_values(by='ia_100000_2week', ascending=False).transpose().values.tolist(),
               fill_color='rgb(179, 179, 179)',
               align='center', height=25))
])
fig.update_layout(width=950, height=750, title='Datos del último día')
py.iplot(fig, filename='CCAA infected table')

En la siguiente tabla se muestran los fallecimientos acumulados, diarios, incrementos respecto al día anterior, en la última semana y por cada 100000 habitantes, ordenados por este último:

In [41]:
fig = go.Figure(data=[go.Table(
    columnwidth=[50, 90, 70, 70, 70, 70, 120],
    header=dict(values=list(['Fecha', 'CCAA', 'Fallecidos totales', 'Fallecidos diarios', 'Incremento', 'Fallecidos 7 días', 'Fallecidos por 100000 habitantes ']),
                fill_color='rgb(124,124,124)',
                align='center', font=dict(color='white')),
    cells=dict(values=ccaa[['fecha', 'ccaa', 'deceased_accumulated', 'deceased', 'deceased_inc', 'deaths_7_days', 'deceased_per_100000']].tail(20).sort_values(by='deceased_per_100000', ascending=False).transpose().values.tolist(),
               fill_color='rgb(179, 179, 179)',
               align='center', height=25))
])
fig.update_layout(width=1200, height=800, title='Datos de fallecidos del último día')
py.iplot(fig, filename='CCAA deaths table')

## Incidéncia acumulada

In [42]:
fig = make_subplots(rows=rows, cols=cols, vertical_spacing=0.05, subplot_titles=ccaa['ccaa'].unique())
i, j = 1, 1
for ca in ccaa['ccaa'].unique():
    fig.add_trace(go.Scatter(x=ccaa.loc[ccaa['ccaa']==ca]['fecha'], y=ccaa.loc[ccaa['ccaa']==ca]['ia_100000_week'], fill='tonexty', name=ca, line_color='darkcyan'), row=i, col=j)
    j+=1
    if j>cols:
        i+=1
        j=1
fig.update_layout(showlegend=False, width=1600, height=1200, title='Incidencia acumulada en una semana para cada CA')
py.iplot(fig, filename='CCAA ia week plot')

In [43]:
fig = make_subplots(rows=rows, cols=cols, vertical_spacing=0.05, subplot_titles=ccaa['ccaa'].unique())
i, j = 1, 1
for ca in ccaa['ccaa'].unique():
    fig.add_trace(go.Scatter(x=ccaa.loc[ccaa['ccaa']==ca]['fecha'], y=ccaa.loc[ccaa['ccaa']==ca]['ia_100000_2week'], fill='tonexty', name=ca, line_color='darkcyan'), row=i, col=j)
    j+=1
    if j>cols:
        i+=1
        j=1
fig.update_layout(showlegend=False, width=1600, height=1200, title='Incidencia acumulada en dos semanas para cada CA')
fig.add_hline(y=150, line_color='red', row='all', col='all')
py.iplot(fig, filename='CCAA ia 2 week plot')

## Curvas casos y fallecidos CCAA

In [49]:
rows=5
cols=4
fig = make_subplots(rows=rows, cols=cols, vertical_spacing=0.05, subplot_titles=ccaa['ccaa'].unique())
i, j = 1, 1
for ca in ccaa['ccaa'].unique():
    fig.add_trace(go.Scatter(x=ccaa.loc[ccaa['ccaa']==ca]['fecha'], y=ccaa.loc[ccaa['ccaa']==ca]['num_casos'], fill='tonexty', name='Casos', line_color='darkcyan'), row=i, col=j)
    fig.add_trace(go.Scatter(x=ccaa.loc[ccaa['ccaa']==ca]['fecha'], y=ccaa.loc[ccaa['ccaa']==ca]['deceased'],  fill='tonexty', name='Fallecidos', line_color='darkcyan'), row=i, col=j)
    j+=1
    if j>cols:
        i+=1
        j=1
fig.update_layout(showlegend=False, width=1600, height=1200, title='# casos diarios para cada CA')
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            type='buttons',
            bgcolor='LightSteelBlue',
            buttons=list([
                dict(label="Casos",
                     method="update",
                     args=[{"visible": [True, False]},
                           {"title": "# casos diarios para cada CA"}]),
                dict(label="Fallecidos",
                     method="update",
                     args=[{"visible": [False, True]},
                           {"title": "# de fallecidos diarios para cada CA"}]),
            ]),
            direction="left",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.4,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ])
py.iplot(fig, filename='CCAA daily dropdown  plot')

## Curvas casos y fallecidos acumulados CCAA

In [48]:
rows=5
cols=4
fig = make_subplots(rows=rows, cols=cols, vertical_spacing=0.05, subplot_titles=ccaa['ccaa'].unique())
i, j = 1, 1
for ca in ccaa['ccaa'].unique():
    fig.add_trace(go.Scatter(x=ccaa.loc[ccaa['ccaa']==ca]['fecha'], y=ccaa.loc[ccaa['ccaa']==ca]['cases_accumulated'], fill='tonexty', name='Casos', line_color='darkcyan'), row=i, col=j)
    fig.add_trace(go.Scatter(x=ccaa.loc[ccaa['ccaa']==ca]['fecha'], y=ccaa.loc[ccaa['ccaa']==ca]['deceased_accumulated'],  fill='tonexty', name='Fallecidos', line_color='darkcyan'), row=i, col=j)
    j+=1
    if j>cols:
        i+=1
        j=1
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            type='buttons',
            bgcolor='LightSteelBlue',
            buttons=list([
                dict(label="Casos",
                     method="update",
                     args=[{"visible": [True, False]},
                           {"title": "# casos acumulado para cada CA"}]),
                dict(label="Fallecidos",
                     method="update",
                     args=[{"visible": [False, True]},
                           {"title": "# de fallecidos acumulado para cada CA"}]),
            ]),
            direction="left",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.4,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ])
fig.update_layout(showlegend=False, width=1600, height=1200, title='# casos acumulado para cada CA')
py.iplot(fig, filename='CCAA accumulated dropdown  plot')

## Evolución de hospitalizados y en UCI

In [50]:
rows=5
cols=4
fig = make_subplots(rows=rows, cols=cols, vertical_spacing=0.05, subplot_titles=ccaa['ccaa'].unique())
i, j = 1, 1
for ca in ccaa['ccaa'].unique():
    fig.add_trace(go.Scatter(x=ccaa.loc[ccaa['ccaa']==ca]['fecha'], y=ccaa.loc[ccaa['ccaa']==ca]['hospitalizated'], fill='tonexty', name='Hospitalizados', line_color='darkcyan'), row=i, col=j)
    fig.add_trace(go.Scatter(x=ccaa.loc[ccaa['ccaa']==ca]['fecha'], y=ccaa.loc[ccaa['ccaa']==ca]['UCI'], visible=False, fill='tonexty', name='UCI', line_color='darkcyan'), row=i, col=j)
    j+=1
    if j>cols:
        i+=1
        j=1
fig.update_layout(showlegend=False, width=1600, height=1200, title='# de hospitalizados diarios para cada CA')
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            type='buttons',
            bgcolor='LightSteelBlue',
            buttons=list([
                dict(label="Hospitalizados",
                     method="update",
                     args=[{"visible": [True, False]},
                           {"title": "# de hospitalizados diarios para cada CA"}]),
                dict(label="UCI",
                     method="update",
                     args=[{"visible": [False, True]},
                           {"title": "# de ingresados en UCI diario para cada CA"}]),
            ]),
            direction="left",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.4,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ])
py.iplot(fig, filename='CCAA daily hospi-UCI dropdown  plot')